In [ ]:
!pip install tensorflow matplotlib numpy scikit-learn seaborn

In [ ]:
# Implement the CNN model for classifying CIFAR10 image dataset by dividing the
# model into following 4 stages:
# a. Loading and preprocessing the image data
# b. Defining the model's architecture
# c. Training the model
# d. Estimating the model's performance

In [ ]:
# a. Import the necessary packages final accuracy should increase with epochs 100 
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

# b. Load the training and testing data
print("Loading data...")
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

print(f"Training data shape: {train_data.shape}")
print(f"Testing data shape: {test_data.shape}")

# Separate features and labels
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values
X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

print(f"X_train shape: {X_train.shape}")
print(f"y_train shape: {y_train.shape}")
print(f"X_test shape: {X_test.shape}")
print(f"y_test shape: {y_test.shape}")

# Check unique labels
unique_labels = np.unique(y_train)
print(f"Unique labels: {unique_labels}")
print(f"Number of classes: {len(unique_labels)}")

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = len(unique_labels)
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(f"y_train shape after encoding: {y_train.shape}")
print(f"y_test shape after encoding: {y_test.shape}")

# c. Define the network architecture using Keras
def create_model(optimizer='adam', learning_rate=0.001):
    model = Sequential([
        Dense(512, activation='relu', input_shape=(3072,)),
        Dropout(0.3),
        Dense(256, activation='relu'),
        Dropout(0.3),
        Dense(128, activation='relu'),
        Dropout(0.2),
        Dense(num_classes, activation='softmax')
    ])
    
    # Choose optimizer
    if optimizer.lower() == 'sgd':
        opt = SGD(learning_rate=learning_rate, momentum=0.9)
    else:
        opt = Adam(learning_rate=learning_rate)
    
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create model with Adam optimizer
print("Creating model with Adam optimizer...")
model_adam = create_model(optimizer='adam', learning_rate=0.001)
model_adam.summary()

# Create model with SGD optimizer for comparison
print("Creating model with SGD optimizer...")
model_sgd = create_model(optimizer='sgd', learning_rate=0.01)

# d. Train the model using SGD/Adam optimizer
print("Training models...")

# Training parameters
batch_size = 32
epochs = 10

# Callbacks for early stopping and reducing learning rate
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_loss',
        patience=15,
        restore_best_weights=True
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7
    )
]

# Train with Adam optimizer
print("Training with Adam optimizer...")
history_adam = model_adam.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Train with SGD optimizer
print("Training with SGD optimizer...")
history_sgd = model_sgd.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# e. Evaluate the network
print("Evaluating models...")

# Evaluate Adam model
test_loss_adam, test_accuracy_adam = model_adam.evaluate(X_test, y_test, verbose=0)
print(f"Adam Optimizer - Test Loss: {test_loss_adam:.4f}, Test Accuracy: {test_accuracy_adam:.4f}")

# Evaluate SGD model
test_loss_sgd, test_accuracy_sgd = model_sgd.evaluate(X_test, y_test, verbose=0)
print(f"SGD Optimizer - Test Loss: {test_loss_sgd:.4f}, Test Accuracy: {test_accuracy_sgd:.4f}")

# Make predictions
y_pred_adam = model_adam.predict(X_test)
y_pred_classes_adam = np.argmax(y_pred_adam, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Classification report
print("\nClassification Report (Adam Optimizer):")
print(classification_report(y_true_classes, y_pred_classes_adam))

# Confusion matrix
cm = confusion_matrix(y_true_classes, y_pred_classes_adam)
plt.figure(figsize=(10, 8))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues')
plt.title('Confusion Matrix - Adam Optimizer')
plt.ylabel('True Label')
plt.xlabel('Predicted Label')
plt.show()

# f. Plot the training loss and accuracy
def plot_training_history(history_adam, history_sgd):
    fig, axes = plt.subplots(2, 2, figsize=(15, 10))
    
    # Adam optimizer plots
    axes[0, 0].plot(history_adam.history['loss'], label='Training Loss')
    axes[0, 0].plot(history_adam.history['val_loss'], label='Validation Loss')
    axes[0, 0].set_title('Adam Optimizer - Loss')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Loss')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(history_adam.history['accuracy'], label='Training Accuracy')
    axes[0, 1].plot(history_adam.history['val_accuracy'], label='Validation Accuracy')
    axes[0, 1].set_title('Adam Optimizer - Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # SGD optimizer plots
    axes[1, 0].plot(history_sgd.history['loss'], label='Training Loss')
    axes[1, 0].plot(history_sgd.history['val_loss'], label='Validation Loss')
    axes[1, 0].set_title('SGD Optimizer - Loss')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(history_sgd.history['accuracy'], label='Training Accuracy')
    axes[1, 1].plot(history_sgd.history['val_accuracy'], label='Validation Accuracy')
    axes[1, 1].set_title('SGD Optimizer - Accuracy')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Accuracy')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

# Plot training history
plot_training_history(history_adam, history_sgd)

# Compare final performance
optimizers = ['Adam', 'SGD']
test_accuracies = [test_accuracy_adam, test_accuracy_sgd]
test_losses = [test_loss_adam, test_loss_sgd]

plt.figure(figsize=(10, 6))
x = np.arange(len(optimizers))
width = 0.35

fig, ax = plt.subplots(figsize=(10, 6))
rects1 = ax.bar(x - width/2, test_accuracies, width, label='Accuracy', color='skyblue')
rects2 = ax.bar(x + width/2, test_losses, width, label='Loss', color='lightcoral')

ax.set_xlabel('Optimizer')
ax.set_ylabel('Score')
ax.set_title('Model Performance Comparison')
ax.set_xticks(x)
ax.set_xticklabels(optimizers)
ax.legend()

# Add value labels on bars
def autolabel(rects):
    for rect in rects:
        height = rect.get_height()
        ax.annotate(f'{height:.4f}',
                    xy=(rect.get_x() + rect.get_width() / 2, height),
                    xytext=(0, 3),
                    textcoords="offset points",
                    ha='center', va='bottom')

autolabel(rects1)
autolabel(rects2)

plt.tight_layout()
plt.show()

# Print final comparison
print("\n" + "="*50)
print("FINAL MODEL COMPARISON")
print("="*50)
print(f"Adam Optimizer:")
print(f"  - Final Test Accuracy: {test_accuracy_adam:.4f}")
print(f"  - Final Test Loss: {test_loss_adam:.4f}")
print(f"  - Training Epochs: {len(history_adam.history['loss'])}")

print(f"\nSGD Optimizer:")
print(f"  - Final Test Accuracy: {test_accuracy_sgd:.4f}")
print(f"  - Final Test Loss: {test_loss_sgd:.4f}")
print(f"  - Training Epochs: {len(history_sgd.history['loss'])}")

# Save the best model
if test_accuracy_adam > test_accuracy_sgd:
    best_model = model_adam
    best_optimizer = "Adam"
    best_accuracy = test_accuracy_adam
else:
    best_model = model_sgd
    best_optimizer = "SGD"
    best_accuracy = test_accuracy_sgd

print(f"\nBest model: {best_optimizer} Optimizer with accuracy: {best_accuracy:.4f}")

# Save the best model
best_model.save('best_cifar10_model.h5')
print("Best model saved as 'best_cifar10_model.h5'")

In [ ]:
# Enhanced version with better architecture and training if doesn't increase try this
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Loading data...")
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Separate features and labels
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values
X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(f"Data shapes - X_train: {X_train.shape}, y_train: {y_train.shape}")

# Improved model architecture
def create_improved_model(optimizer='adam', learning_rate=0.001):
    model = Sequential([
        Dense(1024, activation='relu', input_shape=(3072,), 
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(128, activation='relu'),
        Dropout(0.2),
        
        Dense(num_classes, activation='softmax')
    ])
    
    # Choose optimizer
    if optimizer.lower() == 'sgd':
        opt = SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
    
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create improved models
print("Creating improved models...")
model_adam_improved = create_improved_model(optimizer='adam', learning_rate=0.0005)
model_sgd_improved = create_improved_model(optimizer='sgd', learning_rate=0.01)

model_adam_improved.summary()

# Enhanced training parameters
batch_size = 64
epochs = 10

# Improved callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=20,
        restore_best_weights=True,
        mode='max'
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    )
]

print("Training improved models...")

# Train improved Adam model
print("Training improved Adam model...")
history_adam_improved = model_adam_improved.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Train improved SGD model
print("Training improved SGD model...")
history_sgd_improved = model_sgd_improved.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Evaluate improved models
print("Evaluating improved models...")

# Evaluate Adam model
test_loss_adam_imp, test_accuracy_adam_imp = model_adam_improved.evaluate(X_test, y_test, verbose=0)
print(f"Improved Adam - Test Loss: {test_loss_adam_imp:.4f}, Test Accuracy: {test_accuracy_adam_imp:.4f}")

# Evaluate SGD model
test_loss_sgd_imp, test_accuracy_sgd_imp = model_sgd_improved.evaluate(X_test, y_test, verbose=0)
print(f"Improved SGD - Test Loss: {test_loss_sgd_imp:.4f}, Test Accuracy: {test_accuracy_sgd_imp:.4f}")

# Compare with original results
print("\n" + "="*60)
print("PERFORMANCE COMPARISON: ORIGINAL vs IMPROVED")
print("="*60)
print(f"{'Model':<20} {'Original Acc':<15} {'Improved Acc':<15} {'Improvement':<15}")
print(f"{'-'*60}")
print(f"{'Adam':<20} {0.3745:<15.4f} {test_accuracy_adam_imp:<15.4f} {test_accuracy_adam_imp-0.3745:<15.4f}")
print(f"{'SGD':<20} {0.4253:<15.4f} {test_accuracy_sgd_imp:<15.4f} {test_accuracy_sgd_imp-0.4253:<15.4f}")

# Plot comparison
def plot_comparison(history_orig_adam, history_orig_sgd, history_imp_adam, history_imp_sgd):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Accuracy comparison
    axes[0, 0].plot(history_orig_adam.history['val_accuracy'], label='Original Adam', alpha=0.7)
    axes[0, 0].plot(history_imp_adam.history['val_accuracy'], label='Improved Adam', linewidth=2)
    axes[0, 0].set_title('Adam Optimizer - Validation Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(history_orig_sgd.history['val_accuracy'], label='Original SGD', alpha=0.7)
    axes[0, 1].plot(history_imp_sgd.history['val_accuracy'], label='Improved SGD', linewidth=2)
    axes[0, 1].set_title('SGD Optimizer - Validation Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Loss comparison
    axes[1, 0].plot(history_orig_adam.history['val_loss'], label='Original Adam', alpha=0.7)
    axes[1, 0].plot(history_imp_adam.history['val_loss'], label='Improved Adam', linewidth=2)
    axes[1, 0].set_title('Adam Optimizer - Validation Loss')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(history_orig_sgd.history['val_loss'], label='Original SGD', alpha=0.7)
    axes[1, 1].plot(history_imp_sgd.history['val_loss'], label='Improved SGD', linewidth=2)
    axes[1, 1].set_title('SGD Optimizer - Validation Loss')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

# Make predictions with best improved model
if test_accuracy_adam_imp > test_accuracy_sgd_imp:
    best_improved_model = model_adam_improved
    best_accuracy_imp = test_accuracy_adam_imp
else:
    best_improved_model = model_sgd_improved
    best_accuracy_imp = test_accuracy_sgd_imp

y_pred_improved = best_improved_model.predict(X_test)
y_pred_classes_improved = np.argmax(y_pred_improved, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Improved classification report
print("\nImproved Classification Report:")
print(classification_report(y_true_classes, y_pred_classes_improved))

# Save improved model
best_improved_model.save('improved_cifar10_model.keras')
print(f"\nImproved model saved as 'improved_cifar10_model.keras' with accuracy: {best_accuracy_imp:.4f}")

# Additional: Learning curve analysis
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_adam_improved.history['accuracy'], label='Training Accuracy')
plt.plot(history_adam_improved.history['val_accuracy'], label='Validation Accuracy')
plt.title('Improved Adam - Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history_sgd_improved.history['accuracy'], label='Training Accuracy')
plt.plot(history_sgd_improved.history['val_accuracy'], label='Validation Accuracy')
plt.title('Improved SGD - Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

In [1]:
# Enhanced version with better architecture and training if doesn't increase try this
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import classification_report, confusion_matrix
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import SGD, Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.regularizers import l2
import warnings
warnings.filterwarnings('ignore')

# Set random seeds for reproducibility
np.random.seed(42)
tf.random.set_seed(42)

print("Loading data...")
train_data = pd.read_csv('train_data.csv')
test_data = pd.read_csv('test_data.csv')

# Separate features and labels
X_train = train_data.drop('label', axis=1).values
y_train = train_data['label'].values
X_test = test_data.drop('label', axis=1).values
y_test = test_data['label'].values

# Normalize pixel values to [0, 1]
X_train = X_train.astype('float32') / 255.0
X_test = X_test.astype('float32') / 255.0

# One-hot encode the labels
num_classes = len(np.unique(y_train))
y_train = to_categorical(y_train, num_classes)
y_test = to_categorical(y_test, num_classes)

print(f"Data shapes - X_train: {X_train.shape}, y_train: {y_train.shape}")

# Improved model architecture
def create_improved_model(optimizer='adam', learning_rate=0.001):
    model = Sequential([
        Dense(1024, activation='relu', input_shape=(3072,), 
              kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(512, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.5),
        
        Dense(256, activation='relu', kernel_regularizer=l2(0.001)),
        BatchNormalization(),
        Dropout(0.3),
        
        Dense(128, activation='relu'),
        Dropout(0.2),
        
        Dense(num_classes, activation='softmax')
    ])
    
    # Choose optimizer
    if optimizer.lower() == 'sgd':
        opt = SGD(learning_rate=learning_rate, momentum=0.9, nesterov=True)
    else:
        opt = Adam(learning_rate=learning_rate, beta_1=0.9, beta_2=0.999)
    
    model.compile(
        optimizer=opt,
        loss='categorical_crossentropy',
        metrics=['accuracy']
    )
    
    return model

# Create improved models
print("Creating improved models...")
model_adam_improved = create_improved_model(optimizer='adam', learning_rate=0.0005)
model_sgd_improved = create_improved_model(optimizer='sgd', learning_rate=0.01)

model_adam_improved.summary()

# Enhanced training parameters
batch_size = 64
epochs = 10

# Improved callbacks
callbacks = [
    keras.callbacks.EarlyStopping(
        monitor='val_accuracy',
        patience=20,
        restore_best_weights=True,
        mode='max'
    ),
    keras.callbacks.ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=10,
        min_lr=1e-7,
        verbose=1
    )
]

print("Training improved models...")

# Train improved Adam model
print("Training improved Adam model...")
history_adam_improved = model_adam_improved.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Train improved SGD model
print("Training improved SGD model...")
history_sgd_improved = model_sgd_improved.fit(
    X_train, y_train,
    batch_size=batch_size,
    epochs=epochs,
    validation_data=(X_test, y_test),
    callbacks=callbacks,
    verbose=1
)

# Evaluate improved models
print("Evaluating improved models...")

# Evaluate Adam model
test_loss_adam_imp, test_accuracy_adam_imp = model_adam_improved.evaluate(X_test, y_test, verbose=0)
print(f"Improved Adam - Test Loss: {test_loss_adam_imp:.4f}, Test Accuracy: {test_accuracy_adam_imp:.4f}")

# Evaluate SGD model
test_loss_sgd_imp, test_accuracy_sgd_imp = model_sgd_improved.evaluate(X_test, y_test, verbose=0)
print(f"Improved SGD - Test Loss: {test_loss_sgd_imp:.4f}, Test Accuracy: {test_accuracy_sgd_imp:.4f}")

# Compare with original results
print("\n" + "="*60)
print("PERFORMANCE COMPARISON: ORIGINAL vs IMPROVED")
print("="*60)
print(f"{'Model':<20} {'Original Acc':<15} {'Improved Acc':<15} {'Improvement':<15}")
print(f"{'-'*60}")
print(f"{'Adam':<20} {0.3745:<15.4f} {test_accuracy_adam_imp:<15.4f} {test_accuracy_adam_imp-0.3745:<15.4f}")
print(f"{'SGD':<20} {0.4253:<15.4f} {test_accuracy_sgd_imp:<15.4f} {test_accuracy_sgd_imp-0.4253:<15.4f}")

# Plot comparison
def plot_comparison(history_orig_adam, history_orig_sgd, history_imp_adam, history_imp_sgd):
    fig, axes = plt.subplots(2, 2, figsize=(15, 12))
    
    # Accuracy comparison
    axes[0, 0].plot(history_orig_adam.history['val_accuracy'], label='Original Adam', alpha=0.7)
    axes[0, 0].plot(history_imp_adam.history['val_accuracy'], label='Improved Adam', linewidth=2)
    axes[0, 0].set_title('Adam Optimizer - Validation Accuracy')
    axes[0, 0].set_xlabel('Epoch')
    axes[0, 0].set_ylabel('Accuracy')
    axes[0, 0].legend()
    axes[0, 0].grid(True)
    
    axes[0, 1].plot(history_orig_sgd.history['val_accuracy'], label='Original SGD', alpha=0.7)
    axes[0, 1].plot(history_imp_sgd.history['val_accuracy'], label='Improved SGD', linewidth=2)
    axes[0, 1].set_title('SGD Optimizer - Validation Accuracy')
    axes[0, 1].set_xlabel('Epoch')
    axes[0, 1].set_ylabel('Accuracy')
    axes[0, 1].legend()
    axes[0, 1].grid(True)
    
    # Loss comparison
    axes[1, 0].plot(history_orig_adam.history['val_loss'], label='Original Adam', alpha=0.7)
    axes[1, 0].plot(history_imp_adam.history['val_loss'], label='Improved Adam', linewidth=2)
    axes[1, 0].set_title('Adam Optimizer - Validation Loss')
    axes[1, 0].set_xlabel('Epoch')
    axes[1, 0].set_ylabel('Loss')
    axes[1, 0].legend()
    axes[1, 0].grid(True)
    
    axes[1, 1].plot(history_orig_sgd.history['val_loss'], label='Original SGD', alpha=0.7)
    axes[1, 1].plot(history_imp_sgd.history['val_loss'], label='Improved SGD', linewidth=2)
    axes[1, 1].set_title('SGD Optimizer - Validation Loss')
    axes[1, 1].set_xlabel('Epoch')
    axes[1, 1].set_ylabel('Loss')
    axes[1, 1].legend()
    axes[1, 1].grid(True)
    
    plt.tight_layout()
    plt.show()

# Make predictions with best improved model
if test_accuracy_adam_imp > test_accuracy_sgd_imp:
    best_improved_model = model_adam_improved
    best_accuracy_imp = test_accuracy_adam_imp
else:
    best_improved_model = model_sgd_improved
    best_accuracy_imp = test_accuracy_sgd_imp

y_pred_improved = best_improved_model.predict(X_test)
y_pred_classes_improved = np.argmax(y_pred_improved, axis=1)
y_true_classes = np.argmax(y_test, axis=1)

# Improved classification report
print("\nImproved Classification Report:")
print(classification_report(y_true_classes, y_pred_classes_improved))

# Save improved model
best_improved_model.save('improved_cifar10_model.keras')
print(f"\nImproved model saved as 'improved_cifar10_model.keras' with accuracy: {best_accuracy_imp:.4f}")

# Additional: Learning curve analysis
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history_adam_improved.history['accuracy'], label='Training Accuracy')
plt.plot(history_adam_improved.history['val_accuracy'], label='Validation Accuracy')
plt.title('Improved Adam - Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.subplot(1, 2, 2)
plt.plot(history_sgd_improved.history['accuracy'], label='Training Accuracy')
plt.plot(history_sgd_improved.history['val_accuracy'], label='Validation Accuracy')
plt.title('Improved SGD - Learning Curves')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.show()

Loading data...



KeyboardInterrupt


KeyboardInterrupt

